- soustraction des moyennes de chaque valeur = centrer les données
- calcul des directions principales = variance, direction où les points s'éparpillent le plus, calcul des axes de la variance
- projection = projection sur les nouveaux axes

In [154]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import plotly.express as px
import plotly.graph_objects as go

Import dataset and remove column Id

In [119]:
url = 'https://full-stack-bigdata-datasets.s3.eu-west-3.amazonaws.com/Machine+Learning+non+Supervis%C3%A9/PCA/house_prices.csv'
dataset = pd.read_csv(url)

dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [120]:
dataset = dataset.drop("Id", axis=1)

dataset.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Remove NaN

In [121]:
dataset = dataset.dropna(axis=1)
dataset

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,20,RL,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,70,RL,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,20,RL,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,112,0,0,0,0,4,2010,WD,Normal,142125


Split data into X and y where X contains all columns except SalePrice

In [122]:
X = dataset.drop("SalePrice", axis=1)
y = dataset["SalePrice"]

y

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [123]:
X

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,...,61,0,0,0,0,0,2,2008,WD,Normal
1,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,...,0,0,0,0,0,0,5,2007,WD,Normal
2,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,...,42,0,0,0,0,0,9,2008,WD,Normal
3,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,...,35,272,0,0,0,0,2,2006,WD,Abnorml
4,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,...,84,0,0,0,0,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,...,40,0,0,0,0,0,8,2007,WD,Normal
1456,20,RL,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,...,0,0,0,0,0,0,2,2010,WD,Normal
1457,70,RL,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,...,60,0,0,0,0,2500,5,2010,WD,Normal
1458,20,RL,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,0,112,0,0,0,0,4,2010,WD,Normal


Using select_dtypes from Pandas, filter out all categorical variables.

In [124]:
# Détection automatique des colonnes numériques et catégorielles
numeric_features = [col for col in X.columns if X[col].dtype != 'object']

print("Filter out categorical features from X:", numeric_features)

# autre facon de faire facilement
# X = X.select_dtypes(exclude=['object'])

Filter out categorical features from X: ['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']


In [125]:
X = X[numeric_features]
X

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60,8450,7,5,2003,2003,706,0,150,856,...,548,0,61,0,0,0,0,0,2,2008
1,20,9600,6,8,1976,1976,978,0,284,1262,...,460,298,0,0,0,0,0,0,5,2007
2,60,11250,7,5,2001,2002,486,0,434,920,...,608,0,42,0,0,0,0,0,9,2008
3,70,9550,7,5,1915,1970,216,0,540,756,...,642,0,35,272,0,0,0,0,2,2006
4,60,14260,8,5,2000,2000,655,0,490,1145,...,836,192,84,0,0,0,0,0,12,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,7917,6,5,1999,2000,0,0,953,953,...,460,0,40,0,0,0,0,0,8,2007
1456,20,13175,6,6,1978,1988,790,163,589,1542,...,500,349,0,0,0,0,0,0,2,2010
1457,70,9042,7,9,1941,2006,275,0,877,1152,...,252,0,60,0,0,0,0,2500,5,2010
1458,20,9717,5,6,1950,1996,49,1029,0,1078,...,240,366,0,112,0,0,0,0,4,2010


Split X, y in a train set and test set.

NB: specify random_state=0 and test_size=0.2

In [126]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=0)
print(f"Train set: {X_train.shape[0]} samples, Test set: {X_test.shape[0]} samples")
print("...Done.\n")

Train set: 1168 samples, Test set: 292 samples
...Done.



Import and fit RandomForestRegressor on your train set. Check out your score on X_test

NB: specify random_state=0 and min_samples_split=15

In [127]:
rfr = RandomForestRegressor(random_state=0, min_samples_split=15)
rfr = rfr.fit(X_train, y_train)

rfr

RandomForestRegressor(min_samples_split=15, random_state=0)

In [128]:
score_rfr = rfr.score(X_test, y_test)
print(f"Score of RandomForestRegressor: {score_rfr}")

Score of RandomForestRegressor: 0.8366009574575247


Verify overfitting

In [129]:
score_rfr = rfr.score(X_train, y_train)
print(f"Score of RandomForestRegressor: {score_rfr}")

Score of RandomForestRegressor: 0.945322157767256


Use cross validation to check your model overall performance : display the average score and the standard deviation

NB: you can use cross_val_score from sklearn

In [130]:
val_score = cross_val_score(rfr, X_train, y_train, cv=15)
val_score

array([0.89793354, 0.87144977, 0.9116179 , 0.83155257, 0.49240767,
       0.8451011 , 0.88863053, 0.87486714, 0.88293881, 0.83327676,
       0.84188528, 0.85646259, 0.89240733, 0.88857753, 0.87737137])

In [131]:
print(f'The average cross-validated score is: {val_score.mean():.4f}')
print(f'The standard deviation of score is: {val_score.std():.4f}')

The average cross-validated score is: 0.8458
The standard deviation of score is: 0.0973


Normalize your train set and apply it on your test set

In [132]:
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

X_train_scaled

array([[-0.86836547,  0.10321202,  2.13150648, ..., -0.09258767,
         0.25639595, -0.61356151],
       [-0.86836547, -0.37288066, -0.79485211, ..., -0.09258767,
         0.62339407,  0.88411842],
       [-0.63114155,  0.25891881, -0.79485211, ..., -0.09258767,
         0.62339407,  0.88411842],
       ...,
       [ 0.79220197, -0.15511546, -0.06326246, ..., -0.09258767,
        -0.84459842,  1.63295838],
       [ 1.50387373, -0.69102346,  0.66832719, ..., -0.09258767,
         1.35739032, -1.36240148],
       [ 0.08053021,  0.57762239,  0.66832719, ..., -0.09258767,
        -0.11060217,  1.63295838]])

In [133]:
X_test_scaled

array([[-0.86836547,  2.06347266, -0.06326246, ..., -0.09258767,
        -1.21159655, -0.61356151],
       [-0.15669371, -0.10277704, -0.06326246, ..., -0.09258767,
         0.62339407, -1.36240148],
       [-0.15669371, -0.33409415, -0.79485211, ..., -0.09258767,
         0.25639595,  0.88411842],
       ...,
       [-0.15669371, -0.19343465, -0.06326246, ..., -0.09258767,
         0.62339407, -0.61356151],
       [-0.63114155, -0.49157671, -1.52644176, ..., -0.09258767,
         2.09138657,  0.88411842],
       [ 0.43636609, -0.31680378, -0.79485211, ..., -0.09258767,
        -1.21159655, -0.61356151]])

Apply PCA on your normalized train set.

NB: specify n_components=3

In [134]:
pca = PCA(n_components=3)
X_train_pca = pca.fit_transform(X_train_scaled)

X_train_pca

array([[ 4.328848  , -0.58109309,  0.61174832],
       [-3.23032795, -0.38723925,  1.06219162],
       [-2.37447855, -1.77209463, -0.32566518],
       ...,
       [-0.33843671,  2.87164554,  0.00682469],
       [ 1.33778317, -0.2164039 ,  1.97466599],
       [ 1.62886359,  1.72608668,  1.42988244]])

Visualize your principal components

In [135]:
X_train_pca = pd.DataFrame(X_train_pca)

X_train_pca

,0,1,2
0,4.328848,-0.581093,0.611748
1,-3.230328,-0.387239,1.062192
2,-2.374479,-1.772095,-0.325665
3,3.683356,-2.856152,-0.458296
4,2.951255,-1.151338,1.292822
...,...,...,...
1163,4.067719,-0.132826,-0.203044
1164,-1.703834,-1.313257,0.754986
1165,-0.338437,2.871646,0.006825
1166,1.337783,-0.216404,1.974666


Check the explained variance ratio from your principal components

In [140]:
print(f"Explained Variance ration per PC: {pca.explained_variance_ratio_}")
print(f"Total explained variance ratio: {pca.explained_variance_ratio_.sum()}")

Explained Variance ration per PC: [0.19200008 0.0982849  0.06323249]
Total explained variance ratio: 0.3535174645588455


Fit your RandomForestRegressor on your train set where you applied PCA, check out your score on your test set.

In [141]:
rfr = rfr.fit(X_train_pca, y_train)
rfr

RandomForestRegressor(min_samples_split=15, random_state=0)

In [143]:
X_test_pca = pca.transform(X_test_scaled)

X_test_pca

array([[ 3.90947109e+00, -3.97598271e-01, -4.44113346e+00],
       [-1.96476393e+00,  3.14651636e-03, -2.03535268e+00],
       [-2.84188867e+00,  1.04753204e-01, -1.26982895e+00],
       [ 2.22755531e+00,  1.82486181e+00,  2.59544127e-02],
       [-2.67052127e+00,  2.15538054e+00,  1.96513338e+00],
       [-3.29715949e+00, -1.43089770e+00, -1.02341921e+00],
       [ 2.75573509e+00, -2.48826874e+00,  8.48911047e-01],
       [-1.21043532e+00,  1.73848909e+00, -3.65259205e-01],
       [ 9.51779076e+00,  6.08792533e-01, -4.43157864e+00],
       [-8.11946001e-01,  8.62498294e-01,  3.59542909e+00],
       [ 1.09542058e+00, -4.72273509e-01,  2.65447497e+00],
       [ 8.71243586e-01,  1.60196747e+00, -3.33354032e-01],
       [ 1.77575170e+00,  1.43639803e+00,  7.08218226e-01],
       [-1.76644883e+00, -1.03754361e+00, -5.41536003e-01],
       [-2.17096359e+00, -6.82630953e-01, -1.58280831e-01],
       [-1.88828110e+00, -1.94261218e+00, -1.22979083e+00],
       [ 1.80327634e+00,  4.31101096e-01

In [144]:
score_rfr = rfr.score(X_test_pca, y_test)
print(f"Score of RandomForestRegressor: {score_rfr}")

Score of RandomForestRegressor: 0.7892984722390265


Use cross_val_score again to see your model's overall performance. What can you say about it?

In [145]:
val_score = cross_val_score(rfr, X_train_pca, y_train, cv=15)
val_score

array([0.87155961, 0.87420361, 0.86427673, 0.85771546, 0.50128915,
       0.83204463, 0.86795784, 0.90521158, 0.83055949, 0.82142629,
       0.85123069, 0.88029847, 0.88411867, 0.87750896, 0.85133197])

In [146]:
print(f'The average cross-validated score is: {val_score.mean():.4f}')
print(f'The standard deviation of score is: {val_score.std():.4f}')

The average cross-validated score is: 0.8380
The standard deviation of score is: 0.0926


Finally, use scatter_3d from plotly.express and go.Scatter3d from plotly.graph_objects to visualize a 3D graphs with all Principal Components from train and from test.

In [153]:
fig = px.scatter_3d(X_train_pca, x=0, y=1, z=2)
fig.show()

In [158]:
X_train_pca

,0,1,2
0,4.328848,-0.581093,0.611748
1,-3.230328,-0.387239,1.062192
2,-2.374479,-1.772095,-0.325665
3,3.683356,-2.856152,-0.458296
4,2.951255,-1.151338,1.292822
...,...,...,...
1163,4.067719,-0.132826,-0.203044
1164,-1.703834,-1.313257,0.754986
1165,-0.338437,2.871646,0.006825
1166,1.337783,-0.216404,1.974666


In [159]:
type(X_train_pca)

pandas.core.frame.DataFrame

In [161]:
# Use plotly express to plot train data
fig = px.scatter_3d(X_train_pca, 
                    x=0, 
                    y=1, 
                    z=2)

# Add trace with test data 
fig.add_trace(go.Scatter3d(x=X_test_pca[:, 0], 
                            y=X_test_pca[:, 1], 
                            z=X_test_pca[:, 2],
                            mode="markers",
                            name="test"
                          ))

# Render on notebook
fig.show()

In [162]:
# Ensure X_train_pca and X_test_pca are NumPy arrays or DataFrames
# Convert to NumPy arrays if necessary
X_train_pca = np.array(X_train_pca) if not isinstance(X_train_pca, np.ndarray) else X_train_pca
X_test_pca = np.array(X_test_pca) if not isinstance(X_test_pca, np.ndarray) else X_test_pca

# Check the shapes of the arrays to ensure compatibility
print("X_train_pca shape:", X_train_pca.shape)
print("X_test_pca shape:", X_test_pca.shape)

# Create a 3D scatter plot for the training data
fig = px.scatter_3d(
    x=X_train_pca[:, 0],  # First principal component
    y=X_train_pca[:, 1],  # Second principal component
    z=X_train_pca[:, 2],  # Third principal component
    title="3D Scatter Plot: PCA - Train vs Test Data",
    labels={"x": "Principal Component 1", "y": "Principal Component 2", "z": "Principal Component 3"},
    opacity=0.7  # Transparency for better visualization
)

# Add test data to the plot as a new trace
fig.add_trace(
    go.Scatter3d(
        x=X_test_pca[:, 0],  # First principal component
        y=X_test_pca[:, 1],  # Second principal component
        z=X_test_pca[:, 2],  # Third principal component
        mode="markers",
        name="Test Data",
        marker=dict(size=5, color="red", opacity=0.8)  # Customize marker appearance
    )
)

# Update layout for clarity
fig.update_layout(
    scene=dict(
        xaxis_title="Principal Component 1",
        yaxis_title="Principal Component 2",
        zaxis_title="Principal Component 3"
    ),
    legend=dict(
        title="Dataset",
        itemsizing="constant"
    )
)

# Render the plot
fig.show()

X_train_pca shape: (1168, 3)
X_test_pca shape: (292, 3)


In [163]:
#Example on train set coloring by SalePrice
df_plot = pd.DataFrame(X_train_pca)
df_plot.columns = ['PC1', 'PC2', 'PC3']
df_plot['SalePrice'] = list(y_train)

fig = px.scatter_3d(df_plot, x='PC1', y='PC2', z='PC3', color='SalePrice')
fig.show()

In [166]:
#Example on test set coloring by SalePrice
df_plot = pd.DataFrame(X_test_pca)
df_plot.columns = ['PC1', 'PC2', 'PC3']
df_plot['SalePrice'] = list(y_test)

fig = px.scatter_3d(df_plot, x='PC1', y='PC2', z='PC3', color='SalePrice')
fig.show()